# IMDB movie review (aclImdb) generation with scaled-down GPT 

In this notebook, we'll use a scaled-down GPT model for generating text using **Tensorflow** with the **Keras API**. 

First, the needed imports.

In [ ]:
%matplotlib inline

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import random
import string

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

import keras_nlp

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

print('Using Tensorflow version: {}, and Keras version: {}.'.format(tf.__version__, tf.keras.__version__))

Let's check if we have GPU available.

In [ ]:
use_fp16 = False

gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 0:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    from tensorflow.python.client import device_lib
    for d in device_lib.list_local_devices():
        if d.device_type == 'GPU':
            print('GPU', d.physical_device_desc)
    if use_fp16:
        keras.mixed_precision.set_global_policy("mixed_float16")
else:
    print('No GPU, using CPU instead.')

## IMDB data set

Next we'll load the IMDB data set. First time we may have to download the data, which can take a while.

The dataset contains 50000 movies reviews from the Internet Movie Database, split into 25000 reviews for training and 25000 reviews for testing. Half of the reviews are positive (1) and half are negative (0).

The dataset consists of movie reviews as text files in a directory hierarchy, and we use the `text_dataset_from_directory()` function to create a `tf.data.Dataset()` from the text files.

In [ ]:
DATADIR = "/media/gpu-data/imdb"
BATCH_SIZE = 128
VOCAB_SIZE = 5000
SEQ_LEN = 80  

In [ ]:
text_ds = tf.data.TextLineDataset(DATADIR+"/aclImdb/aclImdb-nobr-100k.txt")
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(BATCH_SIZE)

In [ ]:
print(text_ds.unbatch().take(1).get_single_element())

## Train the tokenizer

In [ ]:
%%time

vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    text_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

## Tokenize data

In [ ]:
# packer adds a start token
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)

def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels

text_ds = text_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)


## GPT model

### Initialization

In [ ]:
EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2

In [ ]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)
# Embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)
# Transformer decoders.
for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)  # Giving one argument only skips cross-attention.
# Output.
outputs = keras.layers.Dense(VOCAB_SIZE)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer="adam", loss=loss_fn, metrics=[perplexity])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True)

### Learning

In [ ]:
%%time

EPOCHS = 6

history = model.fit(text_ds, verbose=2, epochs=EPOCHS)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3))

ax1.plot(history.epoch,history.history['loss'], label='training')
ax1.set_title('loss')
ax1.set_xlabel('epoch')

ax2.plot(history.epoch,history.history['perplexity'], label='training')
ax2.set_title('perplexity')
ax2.set_xlabel('epoch');

### Inference

In [ ]:
#review = ""
#review = "This was a great scary movie which"
#review = "This was the best movie of"
review = "A funny movie"

tokenized_review = np.trim_zeros(tokenizer.tokenize(review).numpy(), 'b')

In [ ]:
tokenized_review = np.insert(tokenized_review, 0, tokenizer.token_to_id("[BOS]"))

In [ ]:
prompt_tokens = tf.convert_to_tensor(tokenized_review)
prompt_tokens

In [ ]:
NUM_TOKENS_TO_GENERATE = 80

#prompt_tokens = tf.convert_to_tensor([tokenizer.token_to_id("[BOS]")])

def token_logits_fn(inputs):
    cur_len = inputs.shape[1]
    output = model(inputs)
    return output[:, cur_len - 1, :]

In [ ]:
output_tokens = keras_nlp.utils.greedy_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Greedy search generated text: \n{txt}\n")


In [ ]:
output_tokens = keras_nlp.utils.random_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Random search generated text: \n{txt}\n")

In [ ]:
output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.5,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")

---
*Run this notebook in Google Colaboratory using [this link](https://colab.research.google.com/github/csc-training/intro-to-dl/blob/master/day1/04-tf2-imdb-rnn.ipynb).*